![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_DistilBertForZeroShotClassification.ipynb)

# Import OpenVINO DistilBertForZeroShotClassification models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting DistilBertForZeroShotClassification models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for DistilBertForZeroShotClassification from DistilBertForZeroShotClassification   and they have to be in `Zero Shot Classification
` category.

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers==4.48.3 optimum[openvino]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.6/342.6 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.7 MB/s eta 0:00:00


[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- To load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. We can use this to import and export OpenVINO models with `from_pretrained` and `save_pretrained`.
- By setting `export=True`, the source model is converted to OpenVINO IR format on the fly.
- We'll use [typeform/distilbert-base-uncased-mnli](https://huggingface.co/typeform/distilbert-base-uncased-mnli) model from HuggingFace as an example and load it as a `OVModelForSequenceClassification`, representing an OpenVINO model.
- In addition to the OVModelForSequenceClassification model, we also need to save the `AutoTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from optimum.intel import OVModelForSequenceClassification
from transformers import AutoTokenizer

MODEL_NAME = "typeform/distilbert-base-uncased-mnli"
EXPORT_PATH = f"ov_models/{MODEL_NAME}"

ov_model = OVModelForSequenceClassification.from_pretrained(MODEL_NAME, export=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

ov_model.save_pretrained(EXPORT_PATH)
tokenizer.save_pretrained(EXPORT_PATH)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/258 [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
The `xla_device` ar

('ov_models/typeform/distilbert-base-uncased-mnli/tokenizer_config.json',
 'ov_models/typeform/distilbert-base-uncased-mnli/special_tokens_map.json',
 'ov_models/typeform/distilbert-base-uncased-mnli/vocab.txt',
 'ov_models/typeform/distilbert-base-uncased-mnli/added_tokens.json',
 'ov_models/typeform/distilbert-base-uncased-mnli/tokenizer.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 262688
-rw-r--r-- 1 root root       753 Jun 21 21:47 config.json
-rw-r--r-- 1 root root 267827340 Jun 21 21:47 openvino_model.bin
-rw-r--r-- 1 root root    197981 Jun 21 21:47 openvino_model.xml
-rw-r--r-- 1 root root       125 Jun 21 21:47 special_tokens_map.json
-rw-r--r-- 1 root root      1278 Jun 21 21:47 tokenizer_config.json
-rw-r--r-- 1 root root    711396 Jun 21 21:47 tokenizer.json
-rw-r--r-- 1 root root    231508 Jun 21 21:47 vocab.txt


- As you can see, we need to move `vocabs.txt` from the tokenizer to assets folder which Spark NLP will look for
- We also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [ ]:
!mkdir {EXPORT_PATH}/assets

labels = ov_model.config.label2id
labels = sorted(labels, key=labels.get)

with open(EXPORT_PATH + '/assets/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

!mv {EXPORT_PATH}/vocab.txt {EXPORT_PATH}/assets

In [6]:
!ls -lR {EXPORT_PATH}

ov_models/typeform/distilbert-base-uncased-mnli:
total 262464
drwxr-xr-x 2 root root      4096 Jun 21 21:47 assets
-rw-r--r-- 1 root root       753 Jun 21 21:47 config.json
-rw-r--r-- 1 root root 267827340 Jun 21 21:47 openvino_model.bin
-rw-r--r-- 1 root root    197981 Jun 21 21:47 openvino_model.xml
-rw-r--r-- 1 root root       125 Jun 21 21:47 special_tokens_map.json
-rw-r--r-- 1 root root      1278 Jun 21 21:47 tokenizer_config.json
-rw-r--r-- 1 root root    711396 Jun 21 21:47 tokenizer.json

ov_models/typeform/distilbert-base-uncased-mnli/assets:
total 232
-rw-r--r-- 1 root root     32 Jun 21 21:47 labels.txt
-rw-r--r-- 1 root root 231508 Jun 21 21:47 vocab.txt


In [7]:
!cat {EXPORT_PATH}/assets/labels.txt

ENTAILMENT
NEUTRAL
CONTRADICTION

Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

## Import and Save DistilBertForZeroShotClassification in Spark NLP


- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [8]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 29.7 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [9]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `DistilBertForZeroShotClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DistilBertForZeroShotClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [10]:
from sparknlp.annotator import DistilBertForZeroShotClassification

zero_shot_classifier = DistilBertForZeroShotClassification.loadSavedModel(
    EXPORT_PATH,
    spark
    )\
    .setInputCols(["document", "token"]) \
    .setOutputCol("class") \
    .setCandidateLabels(["urgent", "mobile", "travel", "movie", "music", "sport", "weather", "technology"])

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [11]:
zero_shot_classifier.write().overwrite().save("./{}_spark_nlp_openvino".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [12]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your DistilBertForZeroShotClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [13]:
! ls -l {MODEL_NAME}_spark_nlp_openvino

total 261792
-rw-r--r-- 1 root root 268066504 Jun 21 21:50 distilbert_classification_openvino
drwxr-xr-x 4 root root      4096 Jun 21 21:50 fields
drwxr-xr-x 2 root root      4096 Jun 21 21:50 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny DistilBertForZeroShotClassification model 😊

In [14]:
zero_shot_classifier_loaded = DistilBertForZeroShotClassification.load("./{}_spark_nlp_openvino".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

You can see what labels were used to train this model via `getClasses` function:

In [15]:
zero_shot_classifier_loaded.getClasses()

['NEUTRAL', 'CONTRADICTION', 'ENTAILMENT']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [17]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols("document") \
    .setOutputCol("token")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    zero_shot_classifier_loaded
])

examples = spark.createDataFrame([
    ["I need help resetting my password as soon as possible."],
    ["The latest update completely broke the app on my Android device."],
    ["I'm extremely satisfied with your customer service. Thank you!"],
    ["Planning a trip to Italy next summer, any recommendations?"],
    ["Just watched a documentary about climate change. It was eye-opening."],
    ["Are there any discounts available on your premium subscription?"],
    ["The delivery of my order is delayed, and I would like to know why."],
    ["Looking forward to the concert next weekend!"],
    ["My laptop keeps overheating after the recent software patch."],
    ["What's the best way to learn Python for data science?"]], ["text"])

model = pipeline.fit(examples)
result = model.transform(examples)

result.select("text", "class.result").show(truncate=False)

+--------------------------------------------------------------------+------------+
|text                                                                |result      |
+--------------------------------------------------------------------+------------+
|I need help resetting my password as soon as possible.              |[sport]     |
|The latest update completely broke the app on my Android device.    |[mobile]    |
|I'm extremely satisfied with your customer service. Thank you!      |[technology]|
|Planning a trip to Italy next summer, any recommendations?          |[travel]    |
|Just watched a documentary about climate change. It was eye-opening.|[weather]   |
|Are there any discounts available on your premium subscription?     |[weather]   |
|The delivery of my order is delayed, and I would like to know why.  |[weather]   |
|Looking forward to the concert next weekend!                        |[music]     |
|My laptop keeps overheating after the recent software patch.        |[techn

That's it! You can now go wild and use hundreds of `DistilBertForZeroShotClassification` models from HuggingFace 🤗 in Spark NLP 🚀
